In [1]:
from Grav_polygon import Gravity_Polygon
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy
from scipy.stats import norm
import tensorflow_probability as tfp
import seaborn as sns
import pandas as pd
import corner 
import math as m
import copy
from HessianMCMC import HessianMCMC

tfb = tfp.bijectors
tfd=tfp.distributions
psd_kernels = tfp.math.psd_kernels

In [2]:
def constant64(i):
    return(tf.constant(i,dtype = tf.float64))

In [3]:
## define some numbers
Number_para = 5
obs_N = 25
number_burnin = 3000
number_sample = 10000
steps_gradient_decent = 2000
rho = 2000
pi = constant64(m.pi) # define PI in Tensorflow form

depth = constant64(-50)
thickness = constant64(10)

Range = constant64([-200.,200.])

In [4]:
tf.random.set_seed(8)

# X- values: uniformly distributed
control_index = tf.linspace(Range[0],Range[1],Number_para)
#control_index with non-even distribution
# control_index = tf.linspace(Range[0],Range[1],Number_para)+tf.random.uniform([Number_para],-20,20,dtype = tf.float64)

# define some points to minimize the artifacts
number_of_fixpoints = 10

fix_point1 = tf.transpose(tf.stack([tf.linspace(Range[0]-200,Range[0]-10,number_of_fixpoints),depth*tf.ones(number_of_fixpoints,dtype = tf.float64)]))
fix_point2 = tf.transpose(tf.stack([tf.linspace(Range[1]+10,Range[1]+200,number_of_fixpoints),depth*tf.ones(number_of_fixpoints,dtype = tf.float64)]))

## define the true z-values
True_position = tf.random.normal([Number_para],depth,10,dtype = tf.float64)
True_position_sin = 20*tf.sin(0.04*control_index)+depth

In [5]:
## train the gp model based on True_position


# define x , z
points = tf.stack([control_index,True_position],axis = -1)


# add some points to eliminate the bondary artifacts
points = tf.concat([tf.concat([fix_point1,points],axis = 0),fix_point2],axis = 0)

## define the GP parameters
amplitude = tfp.util.TransformedVariable(
      2, tfb.Exp(), dtype=tf.float64, name='amplitude')
length_scale = tfp.util.TransformedVariable(
      200, tfb.Exp(), dtype=tf.float64, name='length_scale')
kernel = psd_kernels.ExponentiatedQuadratic(amplitude, length_scale)
    
observation_noise_variance = tfp.util.TransformedVariable(
        np.exp(-.1), tfb.Exp(),dtype = tf.float64, name='observation_noise_variance')

# x- index used to construct GP model
observation_index_points = tf.reshape(points[:,0],[Number_para+2*number_of_fixpoints,1])


# We'll use an unconditioned GP to train the kernel parameters.
gp = tfd.GaussianProcess(
    kernel=kernel,
    index_points=observation_index_points,
    observation_noise_variance=observation_noise_variance)

def GaussianProcess_model(kernel,observation_index_points,_control_index,_control_position):
    '''
        Arguments:
            kernel: trained GP kernal
            
        return:
            GP interpolated model index and model psition
            
    '''
    ## define how many points interpolated between control points
    resolution = 5
    
    
    points = tf.stack([_control_index,_control_position],axis = -1)

    points = tf.concat([tf.concat([fix_point1,points],axis = 0),fix_point2],axis = 0)
    
    # x-index where we want to interpolate
    model_index = tf.expand_dims(tf.linspace(observation_index_points[0,0],observation_index_points[-1,0],resolution*Number_para+4),axis =1)

    gprm = tfd.GaussianProcessRegressionModel(
        kernel=kernel,
        index_points=model_index,
        observation_index_points=observation_index_points,
        observations=points[:,1],
        observation_noise_variance=observation_noise_variance)

    model_position = gprm.mean()

    ## model_complete contains the extended polygon used to do gravity calculation
    model_position_complete = tf.reshape([tf.concat([model_position - thickness,
                                                     tf.reverse(model_position,axis = [-1])],axis = -1)],
                                         shape = [model_position.shape[0]*2,1])

    model_index_complete = tf.concat([model_index,tf.reverse(model_index,axis = [0])],axis = 0)
    
    return model_index_complete,model_position_complete

x_true,z_true = GaussianProcess_model(kernel,observation_index_points,control_index,True_position_sin)



In [8]:
Number_para = 3
tf.random.set_seed(8)

# X- values: uniformly distributed
control_index = tf.linspace(Range[0],Range[1],Number_para)
#control_index with non-even distribution
# control_index = tf.linspace(Range[0],Range[1],Number_para)+tf.random.uniform([Number_para],-20,20,dtype = tf.float64)

# define some points to minimize the artifacts
number_of_fixpoints = 10

fix_point1 = tf.transpose(tf.stack([tf.linspace(Range[0]-200,Range[0]-10,number_of_fixpoints),depth*tf.ones(number_of_fixpoints,dtype = tf.float64)]))
fix_point2 = tf.transpose(tf.stack([tf.linspace(Range[1]+10,Range[1]+200,number_of_fixpoints),depth*tf.ones(number_of_fixpoints,dtype = tf.float64)]))

## define the true z-values

True_position = tf.random.normal([Number_para],depth,10,dtype = tf.float64)
True_position_sin = 20*tf.sin(0.04*control_index)+depth

points = tf.stack([control_index,True_position],axis = -1)


# add some points to eliminate the bondary artifacts
points = tf.concat([tf.concat([fix_point1,points],axis = 0),fix_point2],axis = 0)

## define the GP parameters
amplitude = tfp.util.TransformedVariable(
      2, tfb.Exp(), dtype=tf.float64, name='amplitude')
length_scale = tfp.util.TransformedVariable(
      200, tfb.Exp(), dtype=tf.float64, name='length_scale')
kernel = psd_kernels.ExponentiatedQuadratic(amplitude, length_scale)
    
observation_noise_variance = tfp.util.TransformedVariable(
        np.exp(-.1), tfb.Exp(),dtype = tf.float64, name='observation_noise_variance')

# x- index used to construct GP model
observation_index_points = tf.reshape(points[:,0],[Number_para+2*number_of_fixpoints,1])


# We'll use an unconditioned GP to train the kernel parameters.
gp = tfd.GaussianProcess(
    kernel=kernel,
    index_points=observation_index_points,
    observation_noise_variance=observation_noise_variance)

x_another,z_another = GaussianProcess_model(kernel,observation_index_points,control_index,True_position_sin)



In [31]:
x_true.shape

TensorShape([58, 1])

In [30]:
x_another.shape

TensorShape([38, 1])

In [35]:
def constant64(i):
    return tf.constant(i, dtype=tf.float64)


def divide(a, b):
    """Tensorflow save divide

    Arguments:
        a {[Tensor]} -- [description]
        b {[Tensor]} -- [description]

    Returns:
        [Tensor] -- 
    """
    return tf.math.divide_no_nan(a, b)


pi = constant64(m.pi)


class Gravity_Polygon(tf.Module):
    def __init__(self, obs_N, Range, rho):
        super(Gravity_Polygon, self).__init__()
        self.obs_N = obs_N
        self.Range = Range
        self.rho = constant64(rho)        # density difference   kg/m^3
        self.x_obv = tf.linspace(constant64(-70.), constant64(70.), self.obs_N)
        self.y_obv = tf.zeros(tf.shape(self.x_obv), dtype=tf.float64)

    def A(self, x1, z1, x2, z2):
        numerator = (x2-x1)*(x1*z2-x2*z1)
        denominator = (x2-x1)**2 + (z2-z1)**2
        return divide(numerator, denominator)

    def B(self, x1, z1, x2, z2):
        '''
        x : array, x coordinate
        z : array, z coordinate
        p1, p2 : int, position

        '''
        return divide((z1-z2), (x2-x1))

    def theta_new(self, xn, zn):

        m = tf.atan(divide(zn, xn))

        m = tf.where(m < 0, m + pi, m)

        m = tf.where(m == 0, m + pi/2, m)

        return m

    def Z_new(self, x1, z1, x2, z2):

        # let's do not allow 1) points at origin
        # 2) two points in a sequence have the same x coordinate

        theta1 = self.theta_new(x1, z1)
        theta2 = self.theta_new(x2, z2)

        r1 = (tf.sqrt(x1**2.+z1**2.))
        r2 = (tf.sqrt(x2**2.+z2**2.))

        _A = self.A(x1, z1, x2, z2)
        _B = self.B(x1, z1, x2, z2)

        Z_result = _A*((theta1-theta2)+_B*tf.math.log(divide(r1, r2)))

        return Z_result

    @tf.function
    def calculate_gravity(self, x, z, a):

        x_obv = tf.linspace(self.Range[0], self.Range[1], self.obs_N)
        y_obv = tf.zeros(tf.shape(x_obv), dtype=tf.float64)

        tx = tf.transpose(tf.tile(x, [1, self.obs_N]))

        x_tile = tf.expand_dims(tx, axis=2)

        tz = tf.transpose(tf.tile(z, [1, self.obs_N]))

        z_tile = tf.expand_dims(tz, axis=2)

        x_obv_tile = tf.expand_dims(x_obv, axis=1)
        x_obv_tile = tf.tile(x_obv_tile, [1, a])
        x_obv_tile = tf.reshape(x_obv_tile, [self.obs_N, a, 1])

        z_obv_tile = tf.expand_dims(y_obv, axis=1)
        z_obv_tile = tf.tile(z_obv_tile, [1, a])
        z_obv_tile = tf.reshape(z_obv_tile, [self.obs_N, a, 1])

        _x = x_tile - x_obv_tile
        _z = z_tile - z_obv_tile

        x_ = tf.concat(
            [_x, tf.reshape(_x[:, 0], shape=[self.obs_N, 1, 1])], axis=1)
        z_ = tf.concat(
            [_z, tf.reshape(_z[:, 0], shape=[self.obs_N, 1, 1])], axis=1)

        x1 = x_[:, 0:-1]
        x2 = x_[:, 1:]
        z1 = z_[:, 0:-1]
        z2 = z_[:, 1:]

        # gravitational constant  m^3 kg ^-1 s^-2
        G = constant64(6.67 * 10**(-11))
        gravity = 2*G*self.rho * \
            tf.reduce_sum(-self.Z_new(x1, z1, x2, z2), axis=1)

        print('tracing')
        tf.print('executing')
        return tf.squeeze(gravity)



In [64]:
shape = tf.constant(58,dtype = tf.float32)

In [58]:
gp = Gravity_Polygon(obs_N,Range,rho)
# gp.calculate_gravity(x_true,z_true)
gp.calculate_gravity.get_concrete_function(x=tf.TensorSpec([None,None], tf.float64),
                                           z=tf.TensorSpec([None,None], tf.float64),
                                           a=tf.TensorSpec([]))

tracing


In [59]:
tf.saved_model.save(gp, "./tmp/module_no_signatures")

tracing
INFO:tensorflow:Assets written to: ./tmp/module_no_signatures/assets


In [61]:
gp.calculate_gravity(x_true,z_true,shape)

tracing
executing


<tf.Tensor: shape=(25,), dtype=float64, numpy=
array([6.48689046e-06, 6.59056361e-06, 6.68281247e-06, 6.76448658e-06,
       6.83609410e-06, 6.89795152e-06, 6.95028938e-06, 6.99333892e-06,
       7.02737731e-06, 7.05275764e-06, 7.06990449e-06, 7.07929554e-06,
       7.08142715e-06, 7.07676789e-06, 7.06571680e-06, 7.04855512e-06,
       7.02541336e-06, 6.99623944e-06, 6.96078025e-06, 6.91856785e-06,
       6.86890976e-06, 6.81087825e-06, 6.74328978e-06, 6.66466733e-06,
       6.57317254e-06])>

In [ ]:
gp.calculate_gravity(x_another,z_another)

In [ ]:
gp.calculate_gravity()

In [ ]:
gp1 = Gravity_Polygon(obs_N,Range,rho)

In [ ]:
gp1.calculate_gravity(x_true,z_true)

In [62]:
gp_imported = tf.saved_model.load("./tmp/module_no_signatures")

In [65]:
gp_imported.calculate_gravity(x_true,z_true,shape)

executing


<tf.Tensor: shape=(25,), dtype=float64, numpy=
array([6.48689046e-06, 6.59056361e-06, 6.68281247e-06, 6.76448658e-06,
       6.83609410e-06, 6.89795152e-06, 6.95028938e-06, 6.99333892e-06,
       7.02737731e-06, 7.05275764e-06, 7.06990449e-06, 7.07929554e-06,
       7.08142715e-06, 7.07676789e-06, 7.06571680e-06, 7.04855512e-06,
       7.02541336e-06, 6.99623944e-06, 6.96078025e-06, 6.91856785e-06,
       6.86890976e-06, 6.81087825e-06, 6.74328978e-06, 6.66466733e-06,
       6.57317254e-06])>

Pass shape explicitly

In [66]:
shape1 = tf.constant(38,tf.float32)

In [67]:
gp_imported.calculate_gravity(x_another,z_another,shape1)

executing


<tf.Tensor: shape=(25,), dtype=float64, numpy=
array([6.35966028e-06, 6.44596113e-06, 6.52530074e-06, 6.59950723e-06,
       6.66989774e-06, 6.73753006e-06, 6.80287437e-06, 6.86615724e-06,
       6.92735555e-06, 6.98591225e-06, 7.04108210e-06, 7.09193076e-06,
       7.13698787e-06, 7.17467337e-06, 7.20358327e-06, 7.22163408e-06,
       7.22715824e-06, 7.21906008e-06, 7.19543588e-06, 7.15566251e-06,
       7.09945711e-06, 7.02592061e-06, 6.93551665e-06, 6.82834468e-06,
       6.70395087e-06])>

In [ ]:
class CustomModule(tf.Module):

    def __init__(self):
        super(CustomModule, self).__init__()
        self.v = tf.Variable(1.)

    @tf.function(experimental_relax_shapes = True)
    def __call__(self, x):
        return x * self.v

    @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
    def mutate(self, new_v):
        self.v.assign(new_v)
    
#     @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
#     def t(self,value):
#         return tf.tile(value,[1,self.v])

module = CustomModule()

In [ ]:
module(tf.constant([0.]))
tf.saved_model.save(module, "./tmp/test")

In [ ]:
imported = tf.saved_model.load("./tmp/test")
imported(tf.constant(3.)).mutate()


In [ ]:
imported = tf.saved_model.load("./tmp/test")
assert 3. == imported(tf.constant(3.)).numpy()
imported.mutate(tf.constant(2.))
assert 6. == imported(tf.constant(3.)).numpy()

In [ ]:
module.__call__.get_concrete_function(x=tf.TensorSpec([None], tf.float32))
tf.saved_model.save(module, "./tmp/test")
imported = tf.saved_model.load("./tmp/test")
assert [3.] == imported(tf.constant([3.])).numpy()